In [87]:
import pandas as pd

In [88]:
import glob

path = r"C:\Users\Elias\OneDrive\Desktop\Birth data\*.csv"

csv_files = glob.glob(path)

dfs = [pd.read_csv(f) for f in csv_files]

merged_df = pd.concat(dfs, ignore_index=True)

print(merged_df)


      Type_of_Event  Year_of_Birth Residence_or_Place_of_Birth  \
0             Birth           2022          Place of Residence   
1             Birth           2022          Place of Residence   
2             Birth           2022          Place of Residence   
3             Birth           2022          Place of Residence   
4             Birth           2022          Place of Residence   
...             ...            ...                         ...   
11455         Birth           2018          Place of Residence   
11456         Birth           2018          Place of Residence   
11457         Birth           2018          Place of Residence   
11458         Birth           2018          Place of Residence   
11459         Birth           2018          Place of Residence   

       ZIP_Code_of_Residence   Birthweight_Grams Total_Births  \
0                      91101     500 - 999 grams            0   
1                      91101   1000 - 1499 grams          <11   
2           

In [89]:
#filtering rows with values = 'Unreliable/Unknown' and '<11' for Birthweight_Grams and Total_Births respectively
filtered = merged_df[
    (merged_df["Birthweight_Grams"].str.contains("Unreliable|Unknown", case=False, na=False))
    | (merged_df["Total_Births"] == "<11")
    | (merged_df["Total_Births"] == "0")
]

print(filtered)

      Type_of_Event  Year_of_Birth Residence_or_Place_of_Birth  \
0             Birth           2022          Place of Residence   
1             Birth           2022          Place of Residence   
2             Birth           2022          Place of Residence   
3             Birth           2022          Place of Residence   
8             Birth           2022          Place of Residence   
...             ...            ...                         ...   
11449         Birth           2018          Place of Residence   
11450         Birth           2018          Place of Residence   
11451         Birth           2018          Place of Residence   
11452         Birth           2018          Place of Residence   
11459         Birth           2018          Place of Residence   

       ZIP_Code_of_Residence   Birthweight_Grams Total_Births  \
0                      91101     500 - 999 grams            0   
1                      91101   1000 - 1499 grams          <11   
2           

In [90]:
# removing unreliable/unknown values
merged_df = merged_df[
    ~merged_df["Birthweight_Grams"].str.contains("Unreliable|Unknown", case=False, na=False)
]
# removing last_data_refresh column because it has no use
merged_df = merged_df.drop(columns=["Last_Data_Refresh"])

#replacing values <11 with 5.5
merged_df["Total_Births"] = merged_df["Total_Births"].replace("<11", "5.5")

# converting from string to numeric
merged_df["Total_Births"] = pd.to_numeric(merged_df["Total_Births"], errors="coerce")

In [91]:
print("Remaining Unreliable:", 
      merged_df["Birthweight_Grams"].str.contains("Unreliable|Unknown", na=False).sum())

print("Remaining '<11' or '0':",
      merged_df["Total_Births"].isin(["<11", "0"]).sum())



Remaining Unreliable: 0
Remaining '<11' or '0': 0


In [92]:
import pandas as pd

# List of zip codes to delete bc they are ZIP codes issued to large corporations or government/university areas
zip_codes_to_delete = [
    90009, 90030, 90050, 90051, 90052, 90053, 90054, 90055, 90060, 90070, 90071, 90072,
    90073, 90074, 90075, 90076, 90077, 90078, 90079, 90080, 90081, 90082, 90083, 90084,
    90086, 90087, 90088, 90089, 90090, 90091, 90093, 90095, 90096, 90099, 90103, 90189,
    90202, 90209, 90213, 90223, 90224, 90231, 90233, 90239, 90251, 90261, 90263, 90264,
    90267, 90292, 90294, 90295, 90296, 90306, 90307, 90308, 90309, 90310, 90311, 90312,
    90313, 90397, 90398, 90406, 90407, 90408, 90409, 90410, 90411, 90502, 90506, 90507,
    90508, 90509, 90510, 90607, 90608, 90609, 90610, 90612, 90631, 90637, 90639, 90651,
    90652, 90659, 90661, 90662, 90671, 90702, 90704, 90707, 90711, 90714, 90731, 90733,
    90734, 90744, 90747, 90748, 90749, 90801, 90807, 90809, 90810,90822, 90831, 90832, 90833,
    90834, 90835, 90840, 90842, 90844, 90845, 90846, 90847, 90848, 90853, 90888, 90895,
    90899, 91003, 91009, 91012, 91017, 91021, 91023, 91025, 91031, 91041, 91043, 91046,
    91066, 91077, 91102, 91109, 91110, 91114, 91115, 91116, 91117, 91118, 91121, 91123,
    91124, 91125, 91126, 91129, 91131, 91182, 91184, 91185, 91188, 91189, 91191, 91199,
    91209, 91221, 91222, 91224, 91225, 91226, 91305, 91308, 91309, 91310, 91313, 91322,
    91327, 91328, 91329, 91330, 91333, 91334, 91337, 91341, 91346, 91353, 91357, 91359,
    91363, 91365, 91371, 91372, 91376, 91380, 91382, 91383, 91385, 91386, 91388, 91392,
    91393, 91394, 91395, 91396, 91399, 91404, 91407, 91408, 91409, 91410, 91412, 91413,
    91416, 91426, 91470, 91482, 91495, 91496, 91497, 91499, 91503, 91504, 91507, 91508,
    91510, 91521, 91522, 91523, 91526, 91603, 91608, 91609, 91610, 91611, 91612, 91614,
    91615, 91616, 91617, 91618, 91714, 91715, 91716, 91734, 91735, 91747, 91749, 91759,
    91769, 91778, 91788, 91793, 91795, 91797, 91799, 91802, 91841, 91896, 91899, 93523,
    93539, 93584, 93586, 93590, 93599
]

zip_column = 'ZIP_Code_of_Residence'  

# Delete rows with specified zip codes
merged_df = merged_df[~merged_df[zip_column].isin(zip_codes_to_delete)]

In [93]:
# Total Births Per Zip Code for each year
total_births = (
    merged_df.groupby(["ZIP_Code_of_Residence", "Year_of_Birth"])["Total_Births"]
    .sum()
    .reset_index(name="Total_Births_All")
)


In [94]:
# Calculating Low-Birth Weight Count (<2500 g) for each Zip code
lbw_categories = [
    "500 - 999 grams",
    "1000 - 1499 grams", 
    "1500 - 1999 grams", 
    "2000 - 2499 grams"
]

lbw_df = merged_df[ merged_df["Birthweight_Grams"].isin(lbw_categories) ]

lbw_count = (
    lbw_df.groupby(["ZIP_Code_of_Residence", "Year_of_Birth"])["Total_Births"]
    .sum()
    .reset_index(name="LBW_Count")
)


In [97]:
# Calculating %LBW & High-risk classification
final = total_births.merge(lbw_count, on=["ZIP_Code_of_Residence", "Year_of_Birth"], how="left")
final["LBW_Count"] = final["LBW_Count"].fillna(0)
final["LBW_Pct"] = (final["LBW_Count"] / final["Total_Births_All"]) * 100

# fill with 0
final['LBW_Pct'] = final['LBW_Pct'].fillna(0)
final["High_Risk"] = (final["LBW_Pct"] > 8).astype(int)


In [ ]:
print("Rows:", merged_df.shape[0])
print("Columns:", merged_df.shape[1])
print("\nColumn Names:\n", merged_df.columns.tolist())

In [ ]:
# Save the detailed birth data
merged_df.to_csv("cleaned_birth_data_detailed.csv", index=False)

# Save the summary with LBW calculations
final.to_csv("cleaned_birth_data_summary.csv", index=False)